# Attack Code Testing

## Imports

In [1]:
from adversary import *
from net import Net
import numpy as np
from torch.optim import Optimizer
import torch
from training import training, testing, accuracy
from optimizer import MiniBatchOptimizer
import matplotlib.pyplot as plt
from data_utils import get_mnist, build_data_loaders

## Setup of a Testing Net with Minibatch

In [2]:
train_dataset, test_dataset = get_mnist(normalize=True)

accuracies_naive_fgsm = []
accuracies_naive_proj = []

criterion = torch.nn.CrossEntropyLoss()

epochs = 10
batch_size = 32
learning_rate = 0.01
decreasing_lr = False
use_cuda = True # GPU seems to raise erros on my side
device = torch.device('cuda' if use_cuda and torch.cuda.is_available() else 'cpu')


net_naive = Net().to(device)
train_loader, test_loader = build_data_loaders(train_dataset, test_dataset, batch_size)


In [19]:
epsilons_fgsm = np.arange(0, 1.5, 0.05)

epsilons_proj = []

n = 788 # Thus n is the dimension of each image, which we need to scale the L2 norm for Projected Gradient Descent

# From source https://adversarial-ml-tutorial.org/adversarial_examples/ for L2 PGD
scaler = np.sqrt(2*n)/(np.sqrt(np.pi*np.exp(1)))
for eps in epsilons_fgsm:
    new_eps = eps*scaler
    epsilons_proj.append(new_eps)

In [20]:
epsilons_proj

[0.0,
 0.6792441920337892,
 1.3584883840675783,
 2.0377325761013676,
 2.7169767681351567,
 3.3962209601689457,
 4.075465152202735,
 4.754709344236525,
 5.433953536270313,
 6.113197728304103,
 6.792441920337891,
 7.471686112371681,
 8.15093030440547,
 8.830174496439259,
 9.50941868847305,
 10.188662880506836,
 10.867907072540627,
 11.547151264574417,
 12.226395456608206,
 12.905639648641994,
 13.584883840675783,
 14.264128032709573,
 14.943372224743362,
 15.622616416777152,
 16.30186060881094,
 16.98110480084473,
 17.660348992878518,
 18.339593184912307,
 19.0188373769461,
 19.698081568979887]

## Train Naive Model

In [21]:
mini_opt = MiniBatchOptimizer(net_naive.parameters(), lr=learning_rate, decreasing_lr=decreasing_lr)
losses, acc = training(net_naive, train_loader, mini_opt, criterion, accuracy, epochs=epochs, device=device)
losses, acc = testing(net_naive, test_loader, criterion, accuracy, device=device)


Launching training on cuda
batch 100	loss = 0.0003594	acc = 1.0
batch 200	loss = 0.001926	acc = 1.0
batch 300	loss = 0.002923	acc = 1.0
batch 400	loss = 0.01806	acc = 1.0
batch 500	loss = 0.0073	acc = 1.0
batch 600	loss = 0.002441	acc = 1.0
batch 700	loss = 0.0003793	acc = 1.0
batch 800	loss = 0.004233	acc = 1.0
batch 900	loss = 0.01792	acc = 1.0
batch 1000	loss = 0.0342	acc = 0.9688
batch 1100	loss = 0.001284	acc = 1.0
batch 1200	loss = 0.1825	acc = 0.9375
batch 1300	loss = 0.006582	acc = 1.0
batch 1400	loss = 0.0008634	acc = 1.0
batch 1500	loss = 0.01077	acc = 1.0
batch 1600	loss = 0.001178	acc = 1.0
batch 1700	loss = 0.003245	acc = 1.0
batch 1800	loss = 0.000753	acc = 1.0
epoch 0	avg epoch loss = 0.01786	avg epoch acc = 0.9953
batch 100	loss = 0.0002917	acc = 1.0
batch 200	loss = 0.001662	acc = 1.0
batch 300	loss = 0.002025	acc = 1.0
batch 400	loss = 0.01718	acc = 1.0
batch 500	loss = 0.007298	acc = 1.0
batch 600	loss = 0.001988	acc = 1.0
batch 700	loss = 0.000342	acc = 1.0
batch 80

## Attack Naive Model

Standard FGSM

In [22]:
for eps in epsilons_fgsm:
    loss_attack, acc_attack  = attack(net_naive, criterion, accuracy, test_loader, epsilon=eps, device=device)
    accuracies_naive_fgsm.append(acc_attack)

Epsilon: 0.00	Test Accuracy = 0.972
Epsilon: 0.05	Test Accuracy = 0.966
Epsilon: 0.10	Test Accuracy = 0.958
Epsilon: 0.15	Test Accuracy = 0.949
Epsilon: 0.20	Test Accuracy = 0.935
Epsilon: 0.25	Test Accuracy = 0.914
Epsilon: 0.30	Test Accuracy = 0.886
Epsilon: 0.35	Test Accuracy = 0.846
Epsilon: 0.40	Test Accuracy = 0.792
Epsilon: 0.45	Test Accuracy = 0.707
Epsilon: 0.50	Test Accuracy = 0.609
Epsilon: 0.55	Test Accuracy = 0.492
Epsilon: 0.60	Test Accuracy = 0.371
Epsilon: 0.65	Test Accuracy = 0.262
Epsilon: 0.70	Test Accuracy = 0.181
Epsilon: 0.75	Test Accuracy = 0.120
Epsilon: 0.80	Test Accuracy = 0.073
Epsilon: 0.85	Test Accuracy = 0.044
Epsilon: 0.90	Test Accuracy = 0.025
Epsilon: 0.95	Test Accuracy = 0.015
Epsilon: 1.00	Test Accuracy = 0.011
Epsilon: 1.05	Test Accuracy = 0.009
Epsilon: 1.10	Test Accuracy = 0.008
Epsilon: 1.15	Test Accuracy = 0.008
Epsilon: 1.20	Test Accuracy = 0.008
Epsilon: 1.25	Test Accuracy = 0.010
Epsilon: 1.30	Test Accuracy = 0.011
Epsilon: 1.35	Test Accuracy 

Projected Gradient Descent

In [23]:
for eps in epsilons_proj:
    loss_attack, acc_attack  = projected_attack(net_naive, criterion, accuracy, test_loader, epsilon=eps, alpha=0.1, num_iter=40, device=device)
    accuracies_naive_proj.append(acc_attack)


Epsilon: 0.00	Test Accuracy = 0.972
Epsilon: 0.68	Test Accuracy = 0.972
Epsilon: 1.36	Test Accuracy = 0.972
Epsilon: 2.04	Test Accuracy = 0.972
Epsilon: 2.72	Test Accuracy = 0.971
Epsilon: 3.40	Test Accuracy = 0.971
Epsilon: 4.08	Test Accuracy = 0.971
Epsilon: 4.75	Test Accuracy = 0.971
Epsilon: 5.43	Test Accuracy = 0.971
Epsilon: 6.11	Test Accuracy = 0.971
Epsilon: 6.79	Test Accuracy = 0.970
Epsilon: 7.47	Test Accuracy = 0.970
Epsilon: 8.15	Test Accuracy = 0.970
Epsilon: 8.83	Test Accuracy = 0.969
Epsilon: 9.51	Test Accuracy = 0.965
Epsilon: 10.19	Test Accuracy = 0.953
Epsilon: 10.87	Test Accuracy = 0.921
Epsilon: 11.55	Test Accuracy = 0.852
Epsilon: 12.23	Test Accuracy = 0.732
Epsilon: 12.91	Test Accuracy = 0.599
Epsilon: 13.58	Test Accuracy = 0.521
Epsilon: 14.26	Test Accuracy = 0.463
Epsilon: 14.94	Test Accuracy = 0.414
Epsilon: 15.62	Test Accuracy = 0.372
Epsilon: 16.30	Test Accuracy = 0.333
Epsilon: 16.98	Test Accuracy = 0.299
Epsilon: 17.66	Test Accuracy = 0.272
Epsilon: 18.34	T

## Train Robust Models Version

In [ ]:
robust_net = Net().to(device)
protect_epochs = 10
protect_lr = 0.01
protect_bz = 32
protect_dec_lr = False
prot_train_loader, prot_test_loader = build_data_loaders(train_dataset, test_dataset, protect_bz)
mini_opt_proc = MiniBatchOptimizer(robust_net.parameters(), lr=protect_lr, decreasing_lr=protect_dec_lr)

accuracies_robust_fgsm = []
accuracies_robust_proj = []

In [ ]:
loss_train, acc_train = protected_training(robust_net, prot_train_loader, mini_opt_proc, criterion, accuracy, epochs=protect_epochs, device=device)
loss_test, acc_test = testing(robust_net, test_loader, criterion, accuracy, device=device)

## Attack Robust Model Version

Standard FGSM

In [ ]:
for eps in epsilons_fgsm:
    loss_attack, acc_attack  = attack(robust_net, criterion, accuracy, test_loader, epsilon=eps, device=device)
    accuracies_robust_fgsm.append(acc_attack)

Projected Gradient Descent

In [ ]:
for eps in epsilons_proj:
    loss_attack, acc_attack  = projected_attack(robust_net, criterion, accuracy, test_loader, epsilon=eps, alpha=1e-2, num_iter=40, device=device)
    accuracies_robust_proj.append(acc_attack)


## Comparative Analysis

In [ ]:
epsilons_it = np.arange(0, len(epsilons_pgd), 1)


### Naive Model

In [ ]:
plt.figure(figsize=(5,5))
plt.plot(epsilons_it, accuracies_naive_fgsm, "*-", c='blue', label='Std. FGSM')
plt.plot(epsilons_it, accuracies_naive_proj, "*-", c='orange', label='PGD')

plt.yticks(np.arange(0, 1.1, step=0.1))
plt.xticks(epsilons_it)

plt.title("Naive Model")
plt.xlabel("Epsilon Choices")
plt.ylabel("Accuracy")
plt.legend();

### Robust Mode

In [ ]:
plt.figure(figsize=(5,5))
plt.plot(epsilons_it, accuracies_robust_fgsm, "*-", c='blue', label='Std. FGSM')
plt.plot(epsilons_it, accuracies_robust_proj, "*-", c='orange', label='PGD')

plt.yticks(np.arange(0, 1.1, step=0.1))
plt.xticks(epsilons_it)

plt.title("Robust Model")
plt.xlabel("Epsilon Choices")
plt.ylabel("Accuracy")
plt.legend();

## File output

In [ ]:
print("********************************")
for entry in  accuracies_naive_fgsm:
    print(entry)
print("********************************")
for entry in  accuracies_naive_proj:
    print(entry)
print("********************************")
for entry in  accuracies_robust_fgsm:
    print(entry)
print("********************************")
for entry in  accuracies_robust_proj:
    print(entry)